In [2]:
# ============================================================
# 🕋 Hajj Data Integration & Fuzzy Matching System
# ============================================================
# إعداد ودمج ملفات Excel، ترجمة الأسماء، وتنفيذ مقارنة Fuzzy Matching
# ============================================================

# ==============================
# 📦 Import Libraries
# ==============================
from google.colab import files, data_table
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 🔹 المرحلة 1: دمج ملفات شركات الحج
# ============================================================

# 1️⃣ رفع الملفات من الجهاز
# Ensure files are uploaded before running this cell.
# Use files.upload() in a previous cell or manually upload files to the session storage.
# files_list = list(uploaded.keys()) # This line seems to be causing issues with file access.

# Get a list of all .xlsx files in the current directory
import glob
excel_files = glob.glob('*.xlsx')


# Check if files_list is empty
if not excel_files:
    print("❌ Error: No .xlsx files found in the current directory. Please ensure your Excel files are present.")
else:
    # 2️⃣ قراءة كل ملف ودمجه في DataFrame واحد
    dfs = [pd.read_excel(f) for f in excel_files]
    df_final = pd.concat(dfs, ignore_index=True)
    df_final.columns = df_final.columns.str.strip()

    # 3️⃣ تحديد الأعمدة المهمة للأسماء
    arabic_cols = ['اسم الشركة عربي', 'اسم الشركة', 'شركة الحج', 'اسم مكتب شؤون الحجاج عربي']
    english_cols = ['اسم الشركة انجليزي', 'اسم مكتب شؤون الحجاج انجليزي', 'Hajj Company']

    df_final['Hajj Company (Arabic)'] = None
    df_final['Hajj Company (English)'] = None

    for col in arabic_cols:
        if col in df_final.columns:
            df_final['Hajj Company (Arabic)'] = df_final['Hajj Company (Arabic)'].fillna(df_final[col])

    for col in english_cols:
        if col in df_final.columns:
            df_final['Hajj Company (English)'] = df_final['Hajj Company (English)'].fillna(df_final[col])

    # 4️⃣ دمج المدينة والدولة
    city_columns = ['المدينة', 'City']
    country_columns = ['الدولة', 'Country']

    df_final['City'] = None
    for col in city_columns:
        if col in df_final.columns:
            df_final['City'] = df_final['City'].fillna(df_final[col])

    df_final['Country'] = None
    for col in country_columns:
        if col in df_final.columns:
            df_final['Country'] = df_final['Country'].fillna(df_final[col])

    # 5️⃣ تنظيف الأعمدة الزائدة
    cols_to_remove = ['نوع الباقة', 'السنة بالهجري', 'عدد الحجاج', 'النوع']
    df_final.drop(columns=[c for c in cols_to_remove if c in df_final.columns], inplace=True)

    # 6️⃣ البريد الإلكتروني
    if 'البريد الالكتروني' in df_final.columns:
        df_final.rename(columns={'البريد الالكتروني': 'email'}, inplace=True)
    elif 'Email' in df_final.columns:
        df_final.rename(columns={'Email': 'email'}, inplace=True)
    df_final['email'] = df_final['email'].fillna('غير متوفر')

    # 7️⃣ تحديد السعودية من خلال المدن
    saudi_cities = ['مكة','المدينة','الرياض','جدة','الدمام','الطائف','القطيف','ينبع','تبوك',
                'أبها','نجران','القصيم','الخبر','عرعر','حائل','الباحة','جازان']

    df_final['Country'] = df_final.apply(
    lambda row: 'المملكة العربية السعودية'
    if pd.notna(row['City']) and any(city in str(row['City']) for city in saudi_cities)
    else row['Country'], axis=1
    )

    # 8️⃣ إزالة التكرارات وتنظيف البيانات
    unique_cols = ['Hajj Company (Arabic)', 'Hajj Company (English)', 'City', 'Country', 'email']
    df_final = df_final.sort_values(by='email').drop_duplicates(subset=unique_cols).reset_index(drop=True)

    # 9️⃣ تعبئة القيم المفقودة داخل نفس الشركة
    for col in ['City', 'Country', 'email']:
        df_final[col] = df_final.groupby('Hajj Company (Arabic)')[col].transform(lambda x: x.ffill().bfill())

    # 10️⃣ إكمال القيم المفقودة
    fill_values = {'City': 'غير معروف', 'Country': 'غير معروف', 'email': 'غير متوفر'}
    df_final[['City','Country','email']] = df_final[['City','Country','email']].fillna(fill_values)
    df_final['is_autorized'] = 'Yes'

    # 11️⃣ حفظ النتيجة
    df_final = df_final[['Hajj Company (Arabic)', 'Hajj Company (English)', 'City', 'Country', 'email', 'is_autorized']]
    df_final.to_excel('Hajj_Info_Final.xlsx', index=False)
    print("✅ تم حفظ ملف: Hajj_Info_Final.xlsx")

✅ تم حفظ ملف: Hajj_Info_Final.xlsx


In [3]:
!pip install langdetect googletrans rapidfuzz textdistance -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 49.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 99.8 MB/s eta 0:00:00


In [4]:
!pip install rapidfuzz textdistance sentence-transformers openpyxl -q

In [7]:
# ============================================================
# 🔹 المرحلة 2: ترجمة أسماء الصفحات (الوكالات)
# ============================================================
from langdetect import detect
from googletrans import Translator

df_pages = pd.read_excel('unique_page_names.xlsx')

# كشف اللغة
def detect_lang(text):
    try:
        return detect(str(text))
    except:
        return None

df_pages['detected_language'] = df_pages.iloc[:, 0].apply(detect_lang)
df_pages.rename(columns={df_pages.columns[0]: 'Name'}, inplace=True)

# فصل الأعمدة
df_pages['Name_en'] = df_pages.apply(lambda r: r['Name'] if r['detected_language'] != 'ar' else None, axis=1)
df_pages['Name_ar'] = df_pages.apply(lambda r: r['Name'] if r['detected_language'] == 'ar' else None, axis=1)

# ترجمة المفقود
translator = Translator()

def translate_text(text, src, dest):
    try:
        return translator.translate(str(text), src=src, dest=dest).text
    except:
        return None

df_pages.loc[df_pages['Name_ar'].isna() & df_pages['Name_en'].notna(), 'Name_ar'] = \
    df_pages.loc[df_pages['Name_ar'].isna() & df_pages['Name_en'].notna(), 'Name_en'].apply(lambda x: translate_text(x, 'en', 'ar'))

df_pages.loc[df_pages['Name_en'].isna() & df_pages['Name_ar'].notna(), 'Name_en'] = \
    df_pages.loc[df_pages['Name_en'].isna() & df_pages['Name_ar'].notna(), 'Name_ar'].apply(lambda x: translate_text(x, 'ar', 'en'))

df_pages.drop(columns=['Name', 'detected_language'], inplace=True)
df_pages.rename(columns={'Name_en':'Hajj Company (English)', 'Name_ar':'Hajj Company (Arabic)'}, inplace=True)
df_pages.to_excel('translated_hajj_agencies.xlsx', index=False)
print("✅ تم حفظ ملف: translated_hajj_agencies.xlsx")

# ============================================================
# 🔹 المرحلة 3: مقارنة الشركات (Fuzzy Matching)
# ============================================================

from rapidfuzz import fuzz, process
from difflib import SequenceMatcher
import textdistance
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

print("\n📂 جاري قراءة الملفات...")
file1 = pd.read_excel('/content/translated_hajj_agencies.xlsx')
file2 = pd.read_excel('Hajj_Info_Final.xlsx')

# تنظيف أسماء الشركات الإنجليزية
file2_clean = file2.dropna(subset=['Hajj Company (English)']).copy()
file2_clean = file2_clean[file2_clean['Hajj Company (English)'].str.strip() != '']
# Convert the column to string type to avoid TypeError
file2_clean['Hajj Company (English)'] = file2_clean['Hajj Company (English)'].astype(str)
file2_list = file2_clean['Hajj Company (English)'].str.lower().tolist()

# ------------------------------------------------------------
# 🧩 الدوال الخاصة بكل طريقة
# ------------------------------------------------------------
def method_1_token_set(query, reference_list, threshold=70):
    result = process.extractOne(query.lower().strip(), reference_list, scorer=fuzz.token_set_ratio)
    if result and result[1] >= threshold:
        return result[0], result[1]
    return None, 0

def method_2_ratio(query, reference_list, threshold=70):
    result = process.extractOne(query.lower().strip(), reference_list, scorer=fuzz.ratio)
    if result and result[1] >= threshold:
        return result[0], result[1]
    return None, 0

def method_3_difflib(query, reference_list, threshold=0.7):
    best, score = None, 0
    for ref in reference_list:
        s = SequenceMatcher(None, query.lower().strip(), ref).ratio()
        if s > score:
            best, score = ref, s
    return (best, score*100) if score >= threshold else (None, 0)

def method_4_levenshtein(query, reference_list, threshold=0.7):
    best, score = None, 0
    for ref in reference_list:
        distance = textdistance.levenshtein.distance(query.lower(), ref)
        max_len = max(len(query), len(ref))
        sim = (1 - distance/max_len) if max_len > 0 else 0
        if sim > score:
            best, score = ref, sim
    return (best, score*100) if score >= threshold else (None, 0)

# ------------------------------------------------------------
# 🤖 طريقة الـ Embeddings
# ------------------------------------------------------------
print("🤖 تحميل نموذج Embeddings...")
model = SentenceTransformer('all-MiniLM-L6-v2')
file2_embeddings = model.encode(file2_list, show_progress_bar=True)

def method_5_embeddings(query, ref_embeddings, ref_list, threshold=0.7):
    query_emb = model.encode([query.lower().strip()])
    sims = cosine_similarity(query_emb, ref_embeddings)[0]
    idx = np.argmax(sims)
    score = sims[idx]
    return (ref_list[idx], score*100) if score >= threshold else (None, 0)

# ============================================================
# 🔹 المرحلة 4: تنفيذ المقارنة وحفظ النتائج
# ============================================================
comparison_results = []

print("\n🧪 بدء المقارنة على أول 10 شركات...\n")
for idx, row in file1.head(10).iterrows():
    company = str(row['Hajj Company (English)'])
    if not company.strip():
        continue

    print(f"🔍 {company}")
    result = {'Original': company}

    for i, (func, name) in enumerate([
        (method_1_token_set, "RapidFuzz token_set"),
        (method_2_ratio, "RapidFuzz ratio"),
        (method_3_difflib, "difflib"),
        (method_4_levenshtein, "Levenshtein"),
        (method_5_embeddings, "Embeddings")
    ], 1):
        start = time.time()
        if name == "Embeddings":
            match, score = func(company, file2_embeddings, file2_list)
        else:
            match, score = func(company, file2_list)
        t = (time.time() - start) * 1000
        print(f" {i}) {name:<20} | Match: {match[:40] if match else '-':40} | Score: {score:.1f}% | {t:.2f}ms")
        result[f'{name} Match'] = match
        result[f'{name} Score'] = round(score, 1)
        result[f'{name} Time (ms)'] = round(t, 2)

    comparison_results.append(result)

comparison_df = pd.DataFrame(comparison_results)

# ------------------------------------------------------------
# 🧾 ملخص الأداء
# ------------------------------------------------------------
summary = pd.DataFrame({
    'الطريقة': [
        'RapidFuzz token_set', 'RapidFuzz ratio', 'difflib', 'Levenshtein', 'Embeddings'
    ],
    'متوسط النسبة %': [
        comparison_df[c].mean() for c in comparison_df.columns if 'Score' in c
    ],
    'متوسط الوقت (ms)': [
        comparison_df[c].mean() for c in comparison_df.columns if 'Time' in c
    ],
    'عدد التطابقات': [
        (comparison_df[c] > 0).sum() for c in comparison_df.columns if 'Score' in c
    ]
})

# ------------------------------------------------------------
# 💾 حفظ النتائج
# ------------------------------------------------------------
with pd.ExcelWriter('fuzzy_matching_comparison.xlsx', engine='openpyxl') as writer:
    comparison_df.to_excel(writer, sheet_name='تفاصيل المقارنة', index=False)
    summary.to_excel(writer, sheet_name='ملخص الأداء', index=False)

print("\n✅ تم حفظ ملف المقارنة: fuzzy_matching_comparison.xlsx")
print("\n🎯 الأكثر دقة:", summary.loc[summary['متوسط النسبة %'].idxmax(), 'الطريقة'])
print("⚡ الأسرع:", summary.loc[summary['متوسط الوقت (ms)'].idxmin(), 'الطريقة'])

✅ تم حفظ ملف: translated_hajj_agencies.xlsx

📂 جاري قراءة الملفات...
🤖 تحميل نموذج Embeddings...


Batches:   0%|          | 0/243 [00:00<?, ?it/s]


🧪 بدء المقارنة على أول 10 شركات...

🔍 Salman Faris Ayuba
 1) RapidFuzz token_set  | Match: al faris                                 | Score: 76.9% | 8.03ms
 2) RapidFuzz ratio      | Match: -                                        | Score: 0.0% | 0.84ms
 3) difflib              | Match: -                                        | Score: 0.0% | 436.49ms
 4) Levenshtein          | Match: -                                        | Score: 0.0% | 42.29ms
 5) Embeddings           | Match: -                                        | Score: 0.0% | 45.69ms
🔍 AJWAH Hajj Agency
 1) RapidFuzz token_set  | Match: aljazeera hajj umra and travelling agenc | Score: 78.6% | 27.80ms
 2) RapidFuzz ratio      | Match: mabruran hajj agency                     | Score: 75.7% | 1.49ms
 3) difflib              | Match: mabruran hajj agency                     | Score: 75.7% | 481.77ms
 4) Levenshtein          | Match: mabruran hajj agency                     | Score: 70.0% | 42.63ms
 5) Embeddings           | 